In [0]:
import numpy as np
import tensorflow as tf
import sklearn as sk
import re, os.path, time
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()
!pip install -U numpy
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC,SVC
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

openjdk version "11.0.6" 2020-01-14
OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.18.1)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Database
data_path = '/content/drive/My Drive/Colab Data/Final Project : Salary Prediction /Data'
train_path = 'adult-training.csv'
test_path = 'adult-test.csv'
#Header name
names_col = ['age','workclass','fnlwgt','education','education_num','maritial_status','occupation','relationship','race','gender','capital_gain','capital_loss','hours_per_week','native_country','income_bracket']
#Load training data and testing data
train = pd.read_csv(os.path.join(data_path,train_path),names = names_col,na_values=' ?')
#for col in train.columns:
  #print(col)
pd.set_option('display.max_columns', 5000)
print(train.describe(include = 'all'))

categorical_features = train.select_dtypes(include=['object']).axes[1]
categorical_features = categorical_features.drop('income_bracket',1)
val_train = train['income_bracket']
#print(describe(val_train))
x_train = train.drop('income_bracket',1)
print(categorical_features)
train_num = len(x_train)

test = pd.read_csv(os.path.join(data_path,test_path),names = names_col,na_values=' ?')
y_test = test['income_bracket']
y_test = y_test.map(lambda x:x.strip('.'))
x_test = test.drop('income_bracket',1)

#Imputation
merged = pd.concat([x_train,x_test])
merged = merged.replace(' ','').replace('?', np.nan)
merged_mode = merged[categorical_features].mode()
print(merged_mode)
merged_mode = merged_mode.iloc[0]
temp = []
  
#One-hot encoding
merged_dum = pd.get_dummies(merged)
train_dum = merged_dum[:train_num] #Training Data
test_dum = merged_dum[train_num:]  #Testing Data
#plt.matshow(train.corr())
#plt.show()

#Show correlation between other attributes and income_bracket
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
mapsal = val_train.map({'<=50K' : 0,'>50K' : 1})
mapsal = imp_mean.fit(mapsal)
print("Working...")
print(mapsal)
print("Working...")
mergedWithSal = pd.concat([merged_dum,pd.dataframe(mapsal)])
#mergedWithSal = mergedWithSal.drop('0')
print(mergedWithSal.columns.values)
#print(mergedWithSal.columns)
plt.matshow(mergedWithSal.corr()['income_bracket'][:])
plt.show()


                 age workclass        fnlwgt education  education_num  \
count   32561.000000     30725  3.256100e+04     32561   32561.000000   
unique           NaN         8           NaN        16            NaN   
top              NaN   Private           NaN   HS-grad            NaN   
freq             NaN     22696           NaN     10501            NaN   
mean       38.581647       NaN  1.897784e+05       NaN      10.080679   
std        13.640433       NaN  1.055500e+05       NaN       2.572720   
min        17.000000       NaN  1.228500e+04       NaN       1.000000   
25%        28.000000       NaN  1.178270e+05       NaN       9.000000   
50%        37.000000       NaN  1.783560e+05       NaN      10.000000   
75%        48.000000       NaN  2.370510e+05       NaN      12.000000   
max        90.000000       NaN  1.484705e+06       NaN      16.000000   

            maritial_status       occupation relationship    race gender  \
count                 32561            30718   

ValueError: ignored

In [0]:
model_svm = SVC() #SVM
model_svm.fit(train_dum,val_train)
#scores = cross_val_score(model_svm, train_dum, val_train, cv=10)
#print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_predict = model_svm.predict(test_dum)
result = np.equal(y_predict,y_test)
counter = 0
for i in result:
  if i == True:
    counter +=1
print("Accuracy %f"%(counter/len(result)))

Accuracy 0.798600


In [0]:
probs = model_svm.predict_proba(test_dum)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

AttributeError: ignored

In [0]:
model_rf = RandomForestClassifier()
model_rf.fit(train_dum,val_train)
y_predict_rf = model_rf.predict(test_dum)
result = np.equal(y_predict_rf,y_test)
counter = 0
for i in result:
  if i == True:
    counter +=1
print("Accuracy %f"%(counter/len(result)))